<a href="https://colab.research.google.com/github/CSID-DGU/2020-2-OSSP1-WhatsUp-5/blob/master/model/GPT2_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!git clone https://github.com/SKT-AI/KoGPT2.git

Cloning into 'KoGPT2'...
remote: Enumerating objects: 100, done.
remote: Counting objects: 100% (100/100), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 100 (delta 52), reused 56 (delta 26), pack-reused 0
Receiving objects: 100% (100/100), 548.66 KiB | 30.48 MiB/s, done.
Resolving deltas: 100% (52/52), done.


In [10]:
ls

'='  '=0.1.85'   drive/   KoGPT2/   sample_data/


In [11]:
cd KoGPT2

/content/KoGPT2


In [13]:
ls

imgs/  kogpt2/  LICENSE  README.md  requirements.txt  setup.py


In [14]:
!pip install -r requirements.txt
!pip install .

     |████████████████████████████████| 752.0MB 21kB/s 
     |████████████████████████████████| 675kB 43.0MB/s 
     |████████████████████████████████| 3.8MB 45.9MB/s 
ERROR: torchvision 0.7.0+cu101 has requirement torch==1.6.0, but you'll have torch 1.5.0 which is incompatible.
  Found existing installation: torch 1.6.0+cu101
    Uninstalling torch-1.6.0+cu101:
      Successfully uninstalled torch-1.6.0+cu101
  Found existing installation: tokenizers 0.0.11
    Uninstalling tokenizers-0.0.11:
      Successfully uninstalled tokenizers-0.0.11
  Found existing installation: transformers 2.4.1
    Uninstalling transformers-2.4.1:
      Successfully uninstalled transformers-2.4.1
Processing /content/KoGPT2
  Created wheel for kogpt2: filename=kogpt2-0.1.1-cp36-none-any.whl size=14054 sha256=21bd7fab9c48bdddef9e0019f098ec93eeb5071ff47d757f6fd6951a7f5c0ddc
  Stored in directory: /tmp/pip-ephem-wheel-cache-z56v46c_/wheels/ac/e5/83/e839e6a987261c05b2e32cbd9770007e19f8ea7e2f2f7b9d3c
Successfull

# 허깅페이스의 transformers 라이브러리와 SKT KoGPT2 모델 기반으로 작성된 코드
---환경 설정 때문에 위에서 skt꺼 import 함
--추후에는 해당 git에 있는 requirements.txt만 임포트 하면 될 것이라고 생각함

In [15]:
import os

import numpy as np
import tensorflow as tf

import gluonnlp as nlp
from gluonnlp.data import SentencepieceTokenizer
from transformers import TFGPT2LMHeadModel

from tensorflow.keras.preprocessing.sequence import pad_sequences

from nltk.tokenize import sent_tokenize


모델을 학습하기 위해 필요한 3까지 모듈
1. TFGPT2LMHeadModel: 문장 생성
2. gluonnlp의 SentencepieceTokenizer
3. nlp

In [16]:

class GPT2Model(tf.keras.Model):
    def __init__(self, dir_path):
        super(GPT2Model, self).__init__()
        self.gpt2 = TFGPT2LMHeadModel.from_pretrained(dir_path)
        
    def call(self, inputs):
        return self.gpt2(inputs)[0]

__init__ 함수에서 TFGPT2LMHeadModel을 생성해서 실행할 수 있게 구현함.



```
# self.gpt2(inputs)[0]
```

생성모델을 활용하기 위해서는 vocabulary에 대한 logit 값만 활용하도록 첫 번째 값인 last_hidden_states 출력

In [17]:
!wget https://www.dropbox.com/s/nzfa9xpzm4edp6o/gpt_ckpt.zip -O gpt_ckpt.zip
!unzip -o gpt_ckpt.zip

--2020-10-26 12:07:04--  https://www.dropbox.com/s/nzfa9xpzm4edp6o/gpt_ckpt.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.1, 2620:100:601c:1::a27d:601
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/nzfa9xpzm4edp6o/gpt_ckpt.zip [following]
--2020-10-26 12:07:04--  https://www.dropbox.com/s/raw/nzfa9xpzm4edp6o/gpt_ckpt.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc4d96916498f3f5d3549bd12374.dl.dropboxusercontent.com/cd/0/inline/BB8u70UYAkV8XdaNILU0ZLT7TI9-keDOCvGirjZg8W-y9UgGkXZz_VGN-0MwFbHJXXj2crmAPxG7jPuRov6Gn4K_A8uWDzgCzIFVCaaTUvTEhxO80rCcQO0mdeaGSbhss6c/file# [following]
--2020-10-26 12:07:05--  https://uc4d96916498f3f5d3549bd12374.dl.dropboxusercontent.com/cd/0/inline/BB8u70UYAkV8XdaNILU0ZLT7TI9-keDOCvGirjZg8W-y9UgGkXZz_VGN-0MwFbHJXXj2crmAPxG7jPuRov6Gn4K_A8uWDzgCzIFVCaaTUvTEhxO80r

**학습 파라미터 내려받고 준비하기**

학습된 파라미터가 koGPT2의 경우 huggingface에 모델로 등록돼 있지 않아 파라미터를 다운로드 해야함

In [18]:
BASE_MODEL_PATH = './gpt_ckpt'
gpt_model = GPT2Model(BASE_MODEL_PATH)

모델 리소스 경로를 객체를 생성할 때 인자로 전달하여 학습된 파라미터를 가지는 GPT2 모델 객체 선언

## 사전 학습된 모델을 활용해 언어 생성 결과 확인

단어 하나가 주어지면 문장을 만들어주는 방식으로

In [19]:
BATCH_SIZE = 16
NUM_EPOCHS = 10
MAX_LEN = 30
TOKENIZER_PATH = './gpt_ckpt/gpt2_kor_tokenizer.spiece'

tokenizer = SentencepieceTokenizer(TOKENIZER_PATH)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(TOKENIZER_PATH,
                                               mask_token=None,
                                               sep_token=None,
                                               cls_token=None,
                                               unknown_token='<unk>',
                                               padding_token='<pad>',
                                               bos_token='<s>',
                                               eos_token='</s>')

**토크나이저 생성**

텍스트를 모델에 입력하려면 필요함. 
앞서 불러온 SentencepieceTokenizer와 nlp 모듈의 vocab 활용하여 단어 사전과 토크나이저 정의.


> GPT2의 각 스페셜 토큰의 역할

1.   <unk<unk>> 모르는 단어에 대한 토큰
2.   <pad<pad>> 배치 데이터 길이 맞추는 용도
3.   <s<s>> 문장의 시작을 알림
4.   </s<s>> 문장의 종결을 알림














In [20]:
def tf_top_k_top_p_filtering(logits, top_k=0, top_p=0.0, filter_value=-99999):
    _logits = logits.numpy()
    top_k = min(top_k, logits.shape[-1])  
    if top_k > 0:
        indices_to_remove = logits < tf.math.top_k(logits, top_k)[0][..., -1, None]
        _logits[indices_to_remove] = filter_value

    if top_p > 0.0:
        sorted_logits = tf.sort(logits, direction='DESCENDING')
        sorted_indices = tf.argsort(logits, direction='DESCENDING')
        cumulative_probs = tf.math.cumsum(tf.nn.softmax(sorted_logits, axis=-1), axis=-1)

        sorted_indices_to_remove = cumulative_probs > top_p
        sorted_indices_to_remove = tf.concat([[False], sorted_indices_to_remove[..., :-1]], axis=0)
        indices_to_remove = sorted_indices[sorted_indices_to_remove].numpy().tolist()
        
        _logits[indices_to_remove] = filter_value
    return tf.constant([_logits])


def generate_sent(seed_word, model, max_step=100, greedy=False, top_k=0, top_p=0.):
    sent = seed_word
    toked = tokenizer(sent)
    
    for _ in range(max_step):
        input_ids = tf.constant([vocab[vocab.bos_token],]  + vocab[toked])[None, :] 
        outputs = model(input_ids)[:, -1, :]
        if greedy:
            gen = vocab.to_tokens(tf.argmax(outputs, axis=-1).numpy().tolist()[0])
        else:
            output_logit = tf_top_k_top_p_filtering(outputs[0], top_k=top_k, top_p=top_p)
            gen = vocab.to_tokens(tf.random.categorical(output_logit, 1).numpy().tolist()[0])[0]
        if gen == '</s>':
            break
        sent += gen.replace('▁', ' ')
        toked = tokenizer(sent)

    return sent



```
# generate_sent(seed_word, model, max_step=100, greedy=False, top_k=0, top_p=0.):
```
-see_word: 문장 생성의 시작 단어

-model: 문장 생성을 수행할 모델

-max_step: 생성 횟수를 제한

-greedy: 모델 출력 결과에 대해 유연하게 문장 생성을 해줄 수 있는지 선택할 수 있도록

*   greedy=true: 문장 출력 결과에 대해 가장 확률이 높은 단어만 선택
*   greedy=false: 출력한 단어 가운데 확률 또는 순위가 높은 단어만 선택해 무작위 생성

+) top_k와 top_p 파라미터:  false인 경우 사용. top_k는 확률이 높은 순새대로 k번째까지 필터링. top_p는 일정 확률값 이상인 단어에 대해 필터링



```
#  sent = seed_word
   toked = tokenizer(sent)
```

문장 시작 단어를 변수에 할당하고 토크나이즈


```
# for _ in range(max_step):
        input_ids = tf.constant([vocab[vocab.bos_token],]  + vocab[toked])[None, :] 
        outputs = model(input_ids)[:, -1, :]
```
문장 생성을 할 수 있는 반복문. 토크나이즈된 단어를 인덱스로 변환하고 모델에 입력값으로 넣어 출력값을 받음. 모델의 출력값에 대해서는 문장에서 마지막 단어만 선택

```
# if gen == '</s>':
            break
        sent += gen.replace('▁', ' ')
        toked = tokenizer(sent)
```
생성된 텍스트 토큰이 문장의 끝을 알리는 </s</s>>토큰이면 생성 stop하고 앞서 만들어진 텍스트에 덧붙임






### **테스트-미세 조정 전**

In [21]:
generate_sent('이때', gpt_model, greedy=True)

'이때부터                                                                                                   '

greedy 방식: 확률이 가장 높은 단어만 선택, 학습한 바이어스에 따라 일관된 문장만 출력, 반복되는 단어가 출력되는 결과가 나올 수도 있음

In [22]:
generate_sent('이때', gpt_model, top_k=0, top_p=0.95)

'이때  어떤 생각을 품앗으신가요?'

샘플링 방식: 좀 더 자연스러움

# 소설 텍스트 데이터 전처리하기

미세 조정할 학습 데이터 구성---학습할 데이터는 운수 좋은 날

In [23]:
!git clone https://github.com/NLP-kr/tensorflow-ml-nlp-tf2.git

Cloning into 'tensorflow-ml-nlp-tf2'...
remote: Enumerating objects: 88, done.
remote: Counting objects: 100% (88/88), done.
remote: Compressing objects: 100% (82/82), done.
remote: Total 1531 (delta 48), reused 14 (delta 4), pack-reused 1443
Receiving objects: 100% (1531/1531), 200.94 MiB | 27.07 MiB/s, done.
Resolving deltas: 100% (920/920), done.
Checking out files: 100% (83/83), done.


In [24]:
ls

gpt_ckpt/     imgs/    LICENSE    requirements.txt  tensorflow-ml-nlp-tf2/
gpt_ckpt.zip  kogpt2/  README.md  setup.py


In [25]:
cd tensorflow-ml-nlp-tf2/

/content/KoGPT2/tensorflow-ml-nlp-tf2


In [26]:
cd 7.PRETRAIN_METHOD

/content/KoGPT2/tensorflow-ml-nlp-tf2/7.PRETRAIN_METHOD


In [30]:
ls

7.2.1.bert_finetune_NSMC.ipynb     7.2.5.KorQuAD_EDA.ipynb
7.2.2.bert_finetune_KorNLI.ipynb   7.4.1.gpt2_finetune_LM.ipynb
7.2.2.KorNLI_EDA.ipynb             7.4.2.gpt2_finetune_NSMC.ipynb
7.2.3.bert_finetune_NER.ipynb      7.4.3.gpt2_finetune_KorNLI.ipynb
7.2.3.NER_EDA.ipynb                7.4.4.gpt2_finetune_KorSTS.ipynb
7.2.4.bert_finetune_KorSTS.ipynb   data_in/
7.2.4.KorSTS_EDA.ipynb             README.md
7.2.5.bert_finetune_KorQuAD.ipynb


In [31]:
DATA_IN_PATH = './data_in/KOR/'
TRAIN_DATA_FILE = 'finetune_data.txt'

sents = [s[:-1] for s in open(DATA_IN_PATH + TRAIN_DATA_FILE).readlines()]

학습데이터는 소설 텍스트를 먼저 문장별로 분리해둔 텍스트 데이터

In [32]:
input_data = []
output_data = []

for s in sents:
    tokens = [vocab[vocab.bos_token],]  + vocab[tokenizer(s)] + [vocab[vocab.eos_token],]
    input_data.append(tokens[:-1])
    output_data.append(tokens[1:])

input_data = pad_sequences(input_data, MAX_LEN, value=vocab[vocab.padding_token])
output_data = pad_sequences(output_data, MAX_LEN, value=vocab[vocab.padding_token])

input_data = np.array(input_data, dtype=np.int64)
output_data = np.array(output_data, dtype=np.int64)

토크나이저로 텍스트를 토큰화한 후 입력데이터와 출력 데이터로 구성.
한 문장으로 이루어진 데이터는 문장 시작, 끝에 스페셜 토큰 할당

입력데이터 tokens[:-1]로 맨 앞에서 맨 뒤 직전 토큰까지만 활용

정답데이터를 tokens[1:]로 맨 앞 다음 토큰에서 맨 뒤 토큰까지 활용

pad_sequences 함수를 통해 데이터 패딩 && np.array로 구성하여 학습데이터 준비

In [33]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='accuracy')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, vocab[vocab.padding_token]))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

def accuracy_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, vocab[vocab.padding_token]))
    mask = tf.expand_dims(tf.cast(mask, dtype=pred.dtype), axis=-1)
    pred *= mask    
    acc = train_accuracy(real, pred)

    return tf.reduce_mean(acc)

**손실함수와 정확도 측정 함수**

*loss_object*: 크로스 엔트로피로 손실 값을 측정하기 위한 객체

*train_accuracy*: 정확도 측정을 위한 객체

*loss_function:* 인자로 정답과 예측한 값을 받아서 두 개의 값을 비교해서 손실을 계산하며, real 값 중 0인 값 <PAD<PAD>>는 손실계산에서 뺌
train_accuracy: 정확도를 체크



```
# tf.math.logical_not(tf.math.equal(real, vocab[vocab.padding_token]))
```
정답 real에 포함되는 값 중 vocab[vocab.padding__token]인 것은 <<PAD>PAD>를 의미하는 값. 해당 값들은 True(1)가 되고 이를 제외한 나머지 값들은 False(0)

치환된 요소들에 logical_not 함수를 적용하면 0->1,1->0으로 바뀜

변경된 값을 loss_*=mask에 요소 간에 곱을 해주면 <<PAD>PAD>값은 loss_계산에서 빠짐




In [34]:
gpt_model.compile(loss=loss_function,
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=[accuracy_function])

gpt_model.compile을 통해 loss나 optimizer, metrics 등을 설정해서 미세조정 준비

In [39]:
history = gpt_model.fit(input_data, output_data, 
                    batch_size=BATCH_SIZE, epochs=NUM_EPOCHS,
                    validation_split=0.1)

Epoch 1/10
16/16 [==============================] - 3s 170ms/step - loss: 0.9604 - accuracy_function: 0.4278 - val_loss: 2.6842 - val_accuracy_function: 0.4339
Epoch 2/10
16/16 [==============================] - 3s 170ms/step - loss: 0.8306 - accuracy_function: 0.4423 - val_loss: 2.7775 - val_accuracy_function: 0.4481
Epoch 3/10
16/16 [==============================] - 3s 170ms/step - loss: 0.7311 - accuracy_function: 0.4557 - val_loss: 2.8309 - val_accuracy_function: 0.4615
Epoch 4/10
16/16 [==============================] - 3s 171ms/step - loss: 0.6553 - accuracy_function: 0.4686 - val_loss: 2.8834 - val_accuracy_function: 0.4744
Epoch 5/10
16/16 [==============================] - 3s 173ms/step - loss: 0.5765 - accuracy_function: 0.4813 - val_loss: 2.9562 - val_accuracy_function: 0.4866
Epoch 6/10
16/16 [==============================] - 3s 174ms/step - loss: 0.5058 - accuracy_function: 0.4934 - val_loss: 3.0073 - val_accuracy_function: 0.4984
Epoch 7/10
16/16 [======================

에폭수와 학습 정확도는 비례 but 텍스트 생성과는 반비례

In [40]:
DATA_OUT_PATH = './data_out'
model_name = "tf2_gpt2_finetuned_model"

save_path = os.path.join(DATA_OUT_PATH, model_name)

if not os.path.exists(save_path):
    os.makedirs(save_path)

gpt_model.gpt2.save_pretrained(save_path)

loaded_gpt_model = GPT2Model(save_path)

### **테스트-미세조정 후**

In [41]:
generate_sent('이때', gpt_model, greedy=True)

'이때에                                                                                                   '

In [42]:
generate_sent('이때', gpt_model, top_k=0, top_p=0.95)

'이때 에                                                                                                  '